# Registry Pattern: Plugin Architecture for Extensibility

Registries are **the foundation of Odibi's extensibility** - they allow functions to be discovered, validated, and executed dynamically.

## Part 1: Why Registries?

### The Problem
- User-defined transformations need to be discovered at runtime
- Parameters must be validated before execution
- Metadata (docs, versions, tags) must be tracked
- Functions must be retrieved by name from YAML configs

### The Solution: Registry Pattern
A **central registry** that:
1. Stores functions with decorators
2. Validates parameters using introspection
3. Tracks metadata (versions, categories, tags)
4. Enables discovery and lookup by name

In [ ]:
# Simple registry concept
registry = {}

def register(func):
    """Decorator to register a function."""
    registry[func.__name__] = func
    return func

@register
def add(a, b):
    return a + b

@register
def multiply(a, b):
    return a * b

print(f"Registry: {list(registry.keys())}")
print(f"Calling 'add': {registry['add'](5, 3)}")
print(f"Calling 'multiply': {registry['multiply'](5, 3)}")

## Part 2: FunctionRegistry - The Transform Registry

### Architecture
From `odibi/registry.py`:
```python
class FunctionRegistry:
    _functions: Dict[str, Callable] = {}     # name -> function
    _signatures: Dict[str, Signature] = {}   # name -> signature
```

### Key Features
1. **Class-level storage** (singleton pattern)
2. **Signature tracking** for validation
3. **Parameter validation** before execution
4. **Function discovery** and introspection

In [ ]:
# Simplified FunctionRegistry implementation
from typing import Callable, Dict, Any
import inspect

class FunctionRegistry:
    _functions: Dict[str, Callable] = {}
    _signatures: Dict[str, inspect.Signature] = {}
    
    @classmethod
    def register(cls, func: Callable) -> Callable:
        """Register function and store signature."""
        name = func.__name__
        cls._functions[name] = func
        cls._signatures[name] = inspect.signature(func)
        return func
    
    @classmethod
    def get(cls, name: str) -> Callable:
        """Retrieve registered function."""
        if name not in cls._functions:
            available = ", ".join(cls._functions.keys())
            raise ValueError(f"Function '{name}' not registered. Available: {available}")
        return cls._functions[name]
    
    @classmethod
    def list_functions(cls) -> list[str]:
        """List all registered functions."""
        return list(cls._functions.keys())

# Test it
@FunctionRegistry.register
def clean_text(context, column: str, lowercase: bool = True):
    """Clean text in a column."""
    return f"Cleaned {column}, lowercase={lowercase}"

print(f"Registered: {FunctionRegistry.list_functions()}")
func = FunctionRegistry.get('clean_text')
print(f"Retrieved: {func}")
print(f"Signature: {FunctionRegistry._signatures['clean_text']}")

## Part 3: Parameter Validation with Signatures

The registry uses **inspect.Signature** to validate parameters before execution.

### Validation Logic
1. Extract function parameters (excluding `context`)
2. Check for **missing required** parameters
3. Check for **unexpected** parameters
4. Raise descriptive errors

In [ ]:
# Parameter validation implementation
class FunctionRegistry:
    _functions: Dict[str, Callable] = {}
    _signatures: Dict[str, inspect.Signature] = {}
    
    @classmethod
    def register(cls, func: Callable) -> Callable:
        name = func.__name__
        cls._functions[name] = func
        cls._signatures[name] = inspect.signature(func)
        return func
    
    @classmethod
    def validate_params(cls, name: str, params: Dict[str, Any]) -> None:
        """Validate parameters against function signature."""
        if name not in cls._signatures:
            raise ValueError(f"Function '{name}' not registered")
        
        sig = cls._signatures[name]
        
        # Get function parameters (excluding 'context')
        func_params = {k: v for k, v in sig.parameters.items() if k != 'context'}
        
        # Check for missing required parameters
        missing = []
        for param_name, param in func_params.items():
            if param.default is inspect.Parameter.empty:
                if param_name not in params:
                    missing.append(param_name)
        
        if missing:
            raise ValueError(f"Missing required parameters: {', '.join(missing)}")
        
        # Check for unexpected parameters
        unexpected = set(params.keys()) - set(func_params.keys())
        if unexpected:
            raise ValueError(f"Unexpected parameters: {', '.join(unexpected)}")

# Register a function
@FunctionRegistry.register
def filter_data(context, column: str, threshold: int, inclusive: bool = True):
    """Filter data by threshold."""
    return f"Filter {column} > {threshold}, inclusive={inclusive}"

# Test validation
print("✅ Valid params:")
try:
    FunctionRegistry.validate_params('filter_data', {'column': 'age', 'threshold': 18})
    print("  Validation passed!")
except ValueError as e:
    print(f"  Error: {e}")

print("\n❌ Missing required param:")
try:
    FunctionRegistry.validate_params('filter_data', {'column': 'age'})
except ValueError as e:
    print(f"  Error: {e}")

print("\n❌ Unexpected param:")
try:
    FunctionRegistry.validate_params('filter_data', {
        'column': 'age', 
        'threshold': 18, 
        'extra': 'bad'
    })
except ValueError as e:
    print(f"  Error: {e}")

## Part 4: Function Introspection and Metadata

The registry can **extract rich metadata** from registered functions:
- Parameter names, types, defaults
- Return type annotations
- Docstrings
- Required vs optional params

In [ ]:
# Add introspection to registry
class FunctionRegistry:
    _functions: Dict[str, Callable] = {}
    _signatures: Dict[str, inspect.Signature] = {}
    
    @classmethod
    def register(cls, func: Callable) -> Callable:
        name = func.__name__
        cls._functions[name] = func
        cls._signatures[name] = inspect.signature(func)
        return func
    
    @classmethod
    def get_function_info(cls, name: str) -> Dict[str, Any]:
        """Get detailed function metadata."""
        if name not in cls._functions:
            raise ValueError(f"Function '{name}' not registered")
        
        func = cls._functions[name]
        sig = cls._signatures[name]
        
        # Extract parameter info
        params_info = {}
        for param_name, param in sig.parameters.items():
            if param_name == 'context':
                continue
            
            params_info[param_name] = {
                'required': param.default is inspect.Parameter.empty,
                'default': None if param.default is inspect.Parameter.empty else param.default,
                'annotation': param.annotation if param.annotation != inspect.Parameter.empty else None
            }
        
        return {
            'name': name,
            'docstring': inspect.getdoc(func),
            'parameters': params_info,
            'return_annotation': sig.return_annotation if sig.return_annotation != inspect.Signature.empty else None
        }

# Register and inspect
@FunctionRegistry.register
def aggregate(context, column: str, method: str = 'sum', group_by: str = None) -> dict:
    """Aggregate column using specified method."""
    return {}

info = FunctionRegistry.get_function_info('aggregate')
print(f"Function: {info['name']}")
print(f"\nDocstring: {info['docstring']}")
print(f"\nParameters:")
for name, details in info['parameters'].items():
    req = "REQUIRED" if details['required'] else f"optional (default={details['default']})"
    type_hint = details['annotation'].__name__ if details['annotation'] else 'Any'
    print(f"  {name}: {type_hint} - {req}")
print(f"\nReturns: {info['return_annotation'].__name__}")

## Part 5: @transform Decorator - User-Facing API

The `@transform` decorator provides a **clean interface** for users:

```python
@transform
def my_transform(context, param1: str, param2: int = 10):
    '''Transform description.'''
    data = context.get("input_table")
    # ... logic
    return result
```

### Implementation
- Uses `functools.wraps` to preserve metadata
- Registers function in `FunctionRegistry`
- Returns wrapped function

In [ ]:
from functools import wraps

class FunctionRegistry:
    _functions: Dict[str, Callable] = {}
    _signatures: Dict[str, inspect.Signature] = {}
    
    @classmethod
    def register(cls, func: Callable) -> Callable:
        name = func.__name__
        cls._functions[name] = func
        cls._signatures[name] = inspect.signature(func)
        return func
    
    @classmethod
    def list_functions(cls) -> list[str]:
        return list(cls._functions.keys())

def transform(func: Callable) -> Callable:
    """Decorator to register a transform function."""
    @wraps(func)  # Preserves __name__, __doc__, etc.
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    
    # Register the wrapped function
    FunctionRegistry.register(wrapper)
    return wrapper

# Use the decorator
@transform
def remove_duplicates(context, column: str, keep: str = 'first'):
    """Remove duplicate rows based on column."""
    return f"Dedup on {column}, keep={keep}"

@transform
def fill_missing(context, column: str, value: Any):
    """Fill missing values in column."""
    return f"Fill {column} with {value}"

print(f"Registered transforms: {FunctionRegistry.list_functions()}")
print(f"\nFunction preserved: {remove_duplicates.__name__}")
print(f"Docstring preserved: {remove_duplicates.__doc__.strip()}")

## Part 6: TransformationRegistry - Enhanced Registry with Metadata

From `odibi/transformations/registry.py`:

### Features Beyond FunctionRegistry
1. **Versioning** - Semantic versioning (1.0.0)
2. **Categorization** - Group by type (filtering, aggregation)
3. **Tagging** - Discovery via tags
4. **Docstring validation** - Mandatory documentation
5. **Metadata storage** - Rich function metadata

### Architecture
```python
class TransformationRegistry:
    _transformations: Dict[str, Callable] = {}
    _metadata: Dict[str, Dict[str, Any]] = {}
```

In [ ]:
# TransformationRegistry implementation
from typing import Optional

class TransformationRegistry:
    def __init__(self):
        self._transformations: Dict[str, Callable] = {}
        self._metadata: Dict[str, Dict[str, Any]] = {}
    
    def register(
        self,
        name: str,
        func: Callable,
        version: str = "1.0.0",
        category: Optional[str] = None,
        tags: Optional[list] = None
    ) -> Callable:
        """Register transformation with metadata."""
        # Check for duplicates
        if name in self._transformations:
            raise ValueError(f"Transformation '{name}' already registered")
        
        # Validate docstring
        if not func.__doc__ or len(func.__doc__.strip()) < 10:
            raise ValueError(f"Transformation '{name}' must have docstring (min 10 chars)")
        
        # Store function and metadata
        self._transformations[name] = func
        self._metadata[name] = {
            'version': version,
            'category': category,
            'tags': tags or [],
            'docstring': func.__doc__
        }
        return func
    
    def get(self, name: str) -> Optional[Callable]:
        """Get transformation by name."""
        return self._transformations.get(name)
    
    def get_metadata(self, name: str) -> Optional[Dict[str, Any]]:
        """Get metadata for transformation."""
        return self._metadata.get(name)
    
    def list_all(self) -> Dict[str, Dict[str, Any]]:
        """List all transformations with metadata."""
        return {
            name: {'func': func, **self._metadata[name]}
            for name, func in self._transformations.items()
        }

# Test the registry
registry = TransformationRegistry()

def filter_outliers(df, column, threshold):
    """Remove outliers beyond threshold from specified column."""
    return df

registry.register(
    name='filter_outliers',
    func=filter_outliers,
    version='1.2.0',
    category='filtering',
    tags=['statistics', 'cleaning']
)

metadata = registry.get_metadata('filter_outliers')
print(f"Function: filter_outliers")
print(f"Version: {metadata['version']}")
print(f"Category: {metadata['category']}")
print(f"Tags: {', '.join(metadata['tags'])}")
print(f"Doc: {metadata['docstring'].strip()}")

## Part 7: Docstring Validation and Documentation Enforcement

Odibi **enforces documentation** - every registered transformation must have a meaningful docstring.

### Why?
- Self-documenting code
- Better discovery and understanding
- Generated documentation
- Story generation uses docstrings

In [ ]:
# Docstring validation in action
registry = TransformationRegistry()

def bad_func(df):
    return df

def good_func(df):
    """Apply comprehensive data cleaning and normalization."""
    return df

# This fails
print("❌ Registering function without docstring:")
try:
    registry.register('bad_func', bad_func)
except ValueError as e:
    print(f"  Error: {e}")

# This succeeds
print("\n✅ Registering function with docstring:")
try:
    registry.register('good_func', good_func)
    print("  Registration successful!")
except ValueError as e:
    print(f"  Error: {e}")

## Part 8: @transformation Decorator - Complete Pattern

From `odibi/transformations/decorators.py`:

```python
@transformation("my_transform", category="filtering", tags=["core"])
def my_transform(df, threshold):
    '''Filter records above threshold.'''
    return df[df.value > threshold]
```

### Pattern: Decorator Factory
- Returns a decorator (not direct decoration)
- Accepts metadata parameters
- Registers in global registry

In [ ]:
# Global registry singleton
_global_registry = TransformationRegistry()

def get_registry() -> TransformationRegistry:
    return _global_registry

def transformation(
    name: str,
    version: str = "1.0.0",
    category: Optional[str] = None,
    tags: Optional[list] = None
):
    """Decorator to register a transformation."""
    def decorator(func: Callable) -> Callable:
        # Register in global registry
        registry = get_registry()
        registry.register(
            name=name,
            func=func,
            version=version,
            category=category,
            tags=tags
        )
        return func
    
    return decorator

# Use the decorator
@transformation(
    "normalize_columns",
    version="2.0.1",
    category="preprocessing",
    tags=["ml", "normalization"]
)
def normalize_columns(df, columns: list, method: str = 'minmax'):
    """Normalize specified columns using given method (minmax or zscore)."""
    return f"Normalize {columns} with {method}"

# Retrieve and inspect
registry = get_registry()
all_transforms = registry.list_all()

print("Registered Transformations:")
for name, details in all_transforms.items():
    print(f"\n{name} v{details['version']}")
    print(f"  Category: {details['category']}")
    print(f"  Tags: {', '.join(details['tags'])}")
    print(f"  Doc: {details['docstring'].strip()[:60]}...")

## Part 9: Registry Operations - Discovery and Management

### Core Operations
1. **list_all()** - Get all transformations with metadata
2. **get()** - Retrieve by name
3. **get_metadata()** - Get metadata only
4. **unregister()** - Remove transformation
5. **clear()** - Clear all (for testing)

In [ ]:
class TransformationRegistry:
    def __init__(self):
        self._transformations: Dict[str, Callable] = {}
        self._metadata: Dict[str, Dict[str, Any]] = {}
    
    def register(self, name: str, func: Callable, version: str = "1.0.0",
                 category: Optional[str] = None, tags: Optional[list] = None) -> Callable:
        if name in self._transformations:
            raise ValueError(f"Transformation '{name}' already registered")
        if not func.__doc__ or len(func.__doc__.strip()) < 10:
            raise ValueError(f"Transformation must have docstring")
        self._transformations[name] = func
        self._metadata[name] = {
            'version': version, 'category': category,
            'tags': tags or [], 'docstring': func.__doc__
        }
        return func
    
    def unregister(self, name: str) -> bool:
        """Unregister a transformation."""
        if name in self._transformations:
            del self._transformations[name]
            del self._metadata[name]
            return True
        return False
    
    def clear(self):
        """Clear all registrations (for testing)."""
        self._transformations.clear()
        self._metadata.clear()
    
    def list_all(self) -> Dict[str, Dict[str, Any]]:
        return {
            name: {'func': func, **self._metadata[name]}
            for name, func in self._transformations.items()
        }

# Test operations
registry = TransformationRegistry()

def transform1(df): """First transformation for testing."""; return df
def transform2(df): """Second transformation for testing."""; return df

registry.register('t1', transform1)
registry.register('t2', transform2)

print(f"Registered: {list(registry.list_all().keys())}")

registry.unregister('t1')
print(f"After unregister: {list(registry.list_all().keys())}")

registry.clear()
print(f"After clear: {list(registry.list_all().keys())}")

## Part 10: Testing Registries

### Testing Strategies
1. **Isolation** - Clear registry before each test
2. **Validation** - Test parameter validation logic
3. **Edge cases** - Duplicate names, missing docstrings
4. **Discovery** - Test listing and metadata retrieval

In [ ]:
# Registry testing example
def test_registry():
    """Test suite for TransformationRegistry."""
    registry = TransformationRegistry()
    
    # Test 1: Registration
    def sample_transform(df, col):
        """Sample transformation for testing purposes."""
        return df
    
    registry.register('sample', sample_transform, version='1.0.0')
    assert 'sample' in registry.list_all(), "Registration failed"
    print("✅ Test 1: Registration works")
    
    # Test 2: Duplicate detection
    try:
        registry.register('sample', sample_transform)
        assert False, "Should reject duplicates"
    except ValueError:
        print("✅ Test 2: Duplicate detection works")
    
    # Test 3: Docstring validation
    def no_doc(df): return df
    try:
        registry.register('nodoc', no_doc)
        assert False, "Should require docstring"
    except ValueError:
        print("✅ Test 3: Docstring validation works")
    
    # Test 4: Metadata retrieval
    metadata = registry.get_metadata('sample')
    assert metadata['version'] == '1.0.0', "Metadata incorrect"
    print("✅ Test 4: Metadata retrieval works")
    
    # Test 5: Unregister
    result = registry.unregister('sample')
    assert result == True, "Unregister failed"
    assert 'sample' not in registry.list_all(), "Unregister incomplete"
    print("✅ Test 5: Unregister works")

test_registry()
print("\n✅ All tests passed!")

## Part 11: Real-World Usage in Odibi

### How Registries Enable Odibi's Architecture

1. **YAML Config** → Registry lookup by name
2. **Parameter validation** → Before node execution
3. **Dynamic discovery** → List available transforms
4. **Version management** → Track function evolution
5. **Story generation** → Use docstrings for narrative

### Complete Example

In [ ]:
# Complete real-world example
_global_registry = TransformationRegistry()

def transformation(name: str, version: str = "1.0.0", 
                   category: Optional[str] = None, tags: Optional[list] = None):
    def decorator(func: Callable) -> Callable:
        _global_registry.register(name, func, version, category, tags)
        return func
    return decorator

# User defines transformations
@transformation("clean_names", version="1.0.0", category="cleaning", tags=["text"])
def clean_names(df, column: str):
    """Clean and standardize names in the specified column."""
    return f"Cleaned names in {column}"

@transformation("calculate_age", version="1.0.0", category="feature", tags=["datetime"])
def calculate_age(df, birth_date_col: str, reference_date: str = 'today'):
    """Calculate age from birth date column using reference date."""
    return f"Calculated age from {birth_date_col}"

# YAML config specifies transform by name
node_config = {
    'transform': 'clean_names',
    'params': {'column': 'customer_name'}
}

# Pipeline uses registry to execute
transform_name = node_config['transform']
params = node_config['params']

func = _global_registry.get(transform_name)
metadata = _global_registry.get_metadata(transform_name)

print(f"Executing: {transform_name}")
print(f"Version: {metadata['version']}")
print(f"Category: {metadata['category']}")
print(f"\nResult: {func(None, **params)}")

## Summary: Registry Pattern Benefits

### ✅ What Registries Enable
1. **Dynamic discovery** - Find functions at runtime
2. **Parameter validation** - Catch errors before execution
3. **Extensibility** - Users add custom functions easily
4. **Metadata tracking** - Versions, categories, documentation
5. **Plugin architecture** - Decouple definition from usage

### 🎯 Key Patterns
- **Singleton pattern** - Class-level storage
- **Decorator pattern** - Clean registration API
- **Introspection** - `inspect` module for validation
- **Factory pattern** - Decorator factories for metadata

### 🔧 Odibi's Two Registries
1. **FunctionRegistry** - Simple, signature-based validation
2. **TransformationRegistry** - Enhanced with versioning and metadata

Both work together to make Odibi **infinitely extensible**!